In [43]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.svm import SVR

from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor

import mlflow

In [44]:
# set the dagshub tracking server

mlflow.set_tracking_uri("https://dagshub.com/akashagalaveaaa1/Taxi-Demand-Prediction-System.mlflow")


In [45]:
import dagshub
dagshub.init(repo_owner='akashagalaveaaa1', repo_name='Taxi-Demand-Prediction-System', mlflow=True)

Initialized MLflow to track repo "akashagalaveaaa1/Taxi-Demand-Prediction-System"

Repository akashagalaveaaa1/Taxi-Demand-Prediction-System initialized!

In [46]:
# load the training and test data

train_data_path = "../data/processed/train.csv"
test_data_path = "../data/processed/test.csv"

train_df = pd.read_csv(train_data_path, parse_dates=["tpep_pickup_datetime"]).set_index("tpep_pickup_datetime")

test_df = pd.read_csv(test_data_path, parse_dates=["tpep_pickup_datetime"]).set_index("tpep_pickup_datetime")

train_df

,lag_1,lag_2,lag_3,lag_4,region,total_pickups,avg_pickups,day_of_week
tpep_pickup_datetime,,,,,,,,
2016-01-01 01:00:00,160.0,149.0,120.0,58.0,0,187,161.0,4
2016-01-01 01:15:00,187.0,160.0,149.0,120.0,0,194,175.0,4
2016-01-01 01:30:00,194.0,187.0,160.0,149.0,0,180,177.0,4
2016-01-01 01:45:00,180.0,194.0,187.0,160.0,0,197,185.0,4
2016-01-01 02:00:00,197.0,180.0,194.0,187.0,0,185,185.0,4
...,...,...,...,...,...,...,...,...
2016-02-29 22:45:00,15.0,9.0,11.0,11.0,29,12,12.0,0
2016-02-29 23:00:00,12.0,15.0,9.0,11.0,29,17,14.0,0
2016-02-29 23:15:00,17.0,12.0,15.0,9.0,29,15,14.0,0


In [47]:
# missing value in training data

train_df.isna().sum()

lag_1            0
lag_2            0
lag_3            0
lag_4            0
region           0
total_pickups    0
avg_pickups      0
day_of_week      0
dtype: int64

In [48]:
# missing values in the test data

test_df.isna().sum()

lag_1            0
lag_2            0
lag_3            0
lag_4            0
region           0
total_pickups    0
avg_pickups      0
day_of_week      0
dtype: int64

In [49]:
# make X_train and y_train

X_train = train_df.drop(columns=["total_pickups"])

y_train = train_df["total_pickups"]

In [50]:
X_train.head()

,lag_1,lag_2,lag_3,lag_4,region,avg_pickups,day_of_week
tpep_pickup_datetime,,,,,,,
2016-01-01 01:00:00,160.0,149.0,120.0,58.0,0,161.0,4
2016-01-01 01:15:00,187.0,160.0,149.0,120.0,0,175.0,4
2016-01-01 01:30:00,194.0,187.0,160.0,149.0,0,177.0,4
2016-01-01 01:45:00,180.0,194.0,187.0,160.0,0,185.0,4
2016-01-01 02:00:00,197.0,180.0,194.0,187.0,0,185.0,4


In [51]:
# make X_test and y_test

X_test = test_df.drop(columns=["total_pickups"])

y_test = test_df["total_pickups"]

In [52]:
X_test.head()

,lag_1,lag_2,lag_3,lag_4,region,avg_pickups,day_of_week
tpep_pickup_datetime,,,,,,,
2016-03-01 00:00:00,36.0,44.0,31.0,29.0,0,39.0,1
2016-03-01 00:15:00,41.0,36.0,44.0,31.0,0,37.0,1
2016-03-01 00:30:00,35.0,41.0,36.0,44.0,0,41.0,1
2016-03-01 00:45:00,47.0,35.0,41.0,36.0,0,38.0,1
2016-03-01 01:00:00,34.0,47.0,35.0,41.0,0,35.0,1


In [53]:
from sklearn import set_config

set_config(transform_output="pandas")

In [54]:
# encode the data

encoder = ColumnTransformer([
    ("ohe", OneHotEncoder(drop="first",sparse_output=False), ["region","day_of_week"])
], remainder="passthrough", n_jobs=-1,force_int_remainder_cols=False)

In [55]:
encoder

ColumnTransformer(force_int_remainder_cols=False, n_jobs=-1,
                  remainder='passthrough',
                  transformers=[('ohe',
                                 OneHotEncoder(drop='first',
                                               sparse_output=False),
                                 ['region', 'day_of_week'])])

In [56]:
# encode the train and test data

X_train_encoded = encoder.fit_transform(X_train)
X_test_encoded = encoder.transform(X_test)

In [57]:
import optuna
import tqdm 

In [58]:
# set the experiment

mlflow.set_experiment("Model Selection")

<Experiment: artifact_location='mlflow-artifacts:/03e27324340f498f8c4539333a5763c2', creation_time=1746980667373, experiment_id='0', last_update_time=1746980667373, lifecycle_stage='active', name='Model Selection', tags={}>

In [59]:
def objective(trial):
    # start the child run
    with mlflow.start_run(nested=True) as child:
        
        # model name search space
        list_of_models = ["LR", "RF", "GBR", "XGBR"]
        model_name = trial.suggest_categorical("model_name", list_of_models)
    
        if model_name == "LR":
            model = LinearRegression()
    
        elif model_name == "RF":
            n_estimators_rf = trial.suggest_int("n_estimators_rf",10,100,step=10)
            max_depth_rf = trial.suggest_int("max_depth_rf",3,10)
            model = RandomForestRegressor(n_estimators=n_estimators_rf, 
                                          max_depth=max_depth_rf, 
                                          random_state=42, n_jobs=-1)
    
        elif model_name == "GBR":
            n_estimators_gb = trial.suggest_int("n_estimators_gb",10,100,step=10)
            learning_rate_gb = trial.suggest_float("learning_rate_gb",1e-4,1e-1, log=True)
            model = GradientBoostingRegressor(n_estimators=n_estimators_gb, 
                                              learning_rate=learning_rate_gb,
                                             random_state=42)
    
        elif model_name == "XGBR":
            n_estimators_xgb = trial.suggest_int("n_estimators_xgb",10,100,step=10)
            learning_rate_xgb = trial.suggest_float("learning_rate_xgb",1e-4,1e-1, log=True)
            max_depth_xgb = trial.suggest_int("max_depth_xgb",3,10)
            model = XGBRegressor(n_estimators=n_estimators_xgb,
                                learning_rate=learning_rate_xgb,
                                max_depth=max_depth_xgb)
    
        # log the model name
        mlflow.log_param("model_name",model_name)
        
        # log the model parameters
        mlflow.log_params(model.get_params())
        
        # fit on the data
        model.fit(X_train_encoded,y_train)
    
        # get the predictions
        y_pred = model.predict(X_test_encoded)
    
        # calculate the loss
        loss = mean_absolute_percentage_error(y_test, y_pred)
    
        # log the metric
        mlflow.log_metric("MAPE",loss)
        return loss

In [60]:
# optimize the objective function

with mlflow.start_run(run_name="best_model", nested=True) as parent:

    # create a study object
    study = optuna.create_study(study_name="model_selection", direction="minimize")
    # optimize the objective function
    study.optimize(func=objective, n_trials=50, n_jobs=-1)
    
    # log the best parameters
    mlflow.log_params(study.best_params)
    # log the best error value
    mlflow.log_metric("Best_MAPE", study.best_value)

[I 2025-05-11 22:32:21,448] A new study created in memory with name: model_selection


🏃 View run capable-fawn-488 at: https://dagshub.com/akashagalaveaaa1/Taxi-Demand-Prediction-System.mlflow/#/experiments/0/runs/45cc198f9db04e6687eb268d8465c682
🧪 View experiment at: https://dagshub.com/akashagalaveaaa1/Taxi-Demand-Prediction-System.mlflow/#/experiments/0
🏃 View run melodic-grouse-301 at: https://dagshub.com/akashagalaveaaa1/Taxi-Demand-Prediction-System.mlflow/#/experiments/0/runs/f73f4c51ad3948e19086033697fac67e
🧪 View experiment at: https://dagshub.com/akashagalaveaaa1/Taxi-Demand-Prediction-System.mlflow/#/experiments/0
🏃 View run exultant-shrew-756 at: https://dagshub.com/akashagalaveaaa1/Taxi-Demand-Prediction-System.mlflow/#/experiments/0/runs/b2a38e1392be4ca7b2add8a53e73ee22
🧪 View experiment at: https://dagshub.com/akashagalaveaaa1/Taxi-Demand-Prediction-System.mlflow/#/experiments/0
🏃 View run rambunctious-jay-992 at: https://dagshub.com/akashagalaveaaa1/Taxi-Demand-Prediction-System.mlflow/#/experiments/0/runs/615028fed3fd4ddcbeb85f17756fe927
🧪 View experimen

[I 2025-05-11 22:33:06,428] Trial 4 finished with value: 6.438340187072754 and parameters: {'model_name': 'XGBR', 'n_estimators_xgb': 90, 'learning_rate_xgb': 0.00024120592078432892, 'max_depth_xgb': 6}. Best is trial 4 with value: 6.438340187072754.


🏃 View run big-slug-906 at: https://dagshub.com/akashagalaveaaa1/Taxi-Demand-Prediction-System.mlflow/#/experiments/0/runs/0accc17a030f4b6581e81892ea5bb363
🧪 View experiment at: https://dagshub.com/akashagalaveaaa1/Taxi-Demand-Prediction-System.mlflow/#/experiments/0
🏃 View run upset-grub-870 at: https://dagshub.com/akashagalaveaaa1/Taxi-Demand-Prediction-System.mlflow/#/experiments/0/runs/39b0872c510140ad94965d00c9ff480b
🧪 View experiment at: https://dagshub.com/akashagalaveaaa1/Taxi-Demand-Prediction-System.mlflow/#/experiments/0


[I 2025-05-11 22:33:09,441] Trial 8 finished with value: 0.07934790285463077 and parameters: {'model_name': 'LR'}. Best is trial 8 with value: 0.07934790285463077.
[I 2025-05-11 22:33:10,448] Trial 1 finished with value: 5.961167812347412 and parameters: {'model_name': 'XGBR', 'n_estimators_xgb': 80, 'learning_rate_xgb': 0.0012458595457911774, 'max_depth_xgb': 9}. Best is trial 8 with value: 0.07934790285463077.
[I 2025-05-11 22:33:11,425] Trial 5 finished with value: 0.07934790285463077 and parameters: {'model_name': 'LR'}. Best is trial 8 with value: 0.07934790285463077.


...

🏃 View run bittersweet-grub-673 at: https://dagshub.com/akashagalaveaaa1/Taxi-Demand-Prediction-System.mlflow/#/experiments/0/runs/9df26b629474436ab26ed48ba4c5c45a
🧪 View experiment at: https://dagshub.com/akashagalaveaaa1/Taxi-Demand-Prediction-System.mlflow/#/experiments/0


[I 2025-05-11 22:36:58,461] Trial 45 finished with value: 0.07934790285463077 and parameters: {'model_name': 'LR'}. Best is trial 8 with value: 0.07934790285463077.


🏃 View run adaptable-midge-422 at: https://dagshub.com/akashagalaveaaa1/Taxi-Demand-Prediction-System.mlflow/#/experiments/0/runs/27d5fab23bb149128e7a5bf4399ff20f
🧪 View experiment at: https://dagshub.com/akashagalaveaaa1/Taxi-Demand-Prediction-System.mlflow/#/experiments/0


[I 2025-05-11 22:37:00,445] Trial 47 finished with value: 0.13029011350572198 and parameters: {'model_name': 'RF', 'n_estimators_rf': 10, 'max_depth_rf': 10}. Best is trial 8 with value: 0.07934790285463077.
[I 2025-05-11 22:37:01,430] Trial 48 finished with value: 0.07934790285463077 and parameters: {'model_name': 'LR'}. Best is trial 8 with value: 0.07934790285463077.


🏃 View run best_model at: https://dagshub.com/akashagalaveaaa1/Taxi-Demand-Prediction-System.mlflow/#/experiments/0/runs/4b0ecb279b4d436db637cd5831ada58d
🧪 View experiment at: https://dagshub.com/akashagalaveaaa1/Taxi-Demand-Prediction-System.mlflow/#/experiments/0


In [61]:
# best value

study.best_value

0.07934790285463077

In [62]:
# best parameters

study.best_params

{'model_name': 'LR'}

In [63]:
# model value counts

study.trials_dataframe()['params_model_name'].value_counts()

params_model_name
LR      23
XGBR    12
GBR      9
RF       4
Name: count, dtype: int64

In [68]:
# train the linear regression model

lr = LinearRegression()

lr.fit(X_train_encoded, y_train)

# get predictions
y_pred_train = lr.predict(X_train_encoded) 
y_pred_test = lr.predict(X_test_encoded)

# loss

mape_train = mean_absolute_percentage_error(y_train, y_pred_train)
mape_test = mean_absolute_percentage_error(y_test, y_pred_test)

print("The training error is ", mape_train)
print("The test error is ", mape_test)

The training error is  0.0877801330456651
The test error is  0.07934790285463077


In [69]:
lr.coef_

array([-2.33737604,  0.71512405, -0.55601505, -1.25311068, -3.20463231,
       -0.86685973, -2.79925402, -3.62516859,  0.41386463, -2.9376376 ,
       -1.97624678, -3.75050442,  0.51806283, -2.54033388, -2.43297463,
        0.47632075,  0.61254786, -4.7417372 , -2.03077217, -1.26960984,
       -4.03690273, -2.08863167, -1.0414428 ,  0.73561736, -0.99999442,
       -0.85944985, -2.43098478,  0.67112238,  0.57385071, -0.11719951,
       -0.28045898, -0.37180749, -0.5238324 , -0.4233113 , -0.34045774,
       -0.54170892, -0.36264553, -0.2493965 , -0.31905518,  2.4912456 ])

In [70]:
def tune_ridge(trial):
    # hyperparameter space
    alpha = trial.suggest_float("alpha",30,100)
    
    # make the model object
    ridge = Ridge(alpha=alpha, random_state=42)
    
    # train the model
    ridge.fit(X_train_encoded, y_train)
    
    # get predictions
    y_pred = ridge.predict(X_test_encoded)
    
    # calculate loss
    loss = mean_absolute_percentage_error(y_test, y_pred)

    return loss
        

In [71]:
# create study

study = optuna.create_study(study_name="tune_model", direction="minimize")

[I 2025-05-11 22:42:07,539] A new study created in memory with name: tune_model


In [72]:
# optimize

study.optimize(func=tune_ridge, n_trials=100, n_jobs=-1, show_progress_bar=True)

  0%|          | 0/100 [00:00<?, ?it/s]

[I 2025-05-11 22:42:12,124] Trial 7 finished with value: 0.07918731340757482 and parameters: {'alpha': 82.35288129589719}. Best is trial 1 with value: 0.07918274413809458.
[I 2025-05-11 22:42:12,127] Trial 1 finished with value: 0.07918274413809458 and parameters: {'alpha': 85.64668690776647}. Best is trial 1 with value: 0.07918274413809458.
[I 2025-05-11 22:42:12,345] Trial 9 finished with value: 0.0791850962110863 and parameters: {'alpha': 83.94035572728858}. Best is trial 1 with value: 0.07918274413809458.
[I 2025-05-11 22:42:12,431] Trial 8 finished with value: 0.07922152819872504 and parameters: {'alpha': 59.54826345670036}. Best is trial 1 with value: 0.07918274413809458.
[I 2025-05-11 22:42:12,433] Trial 2 finished with value: 0.0792245564542275 and parameters: {'alpha': 57.684611192666495}. Best is trial 1 with value: 0.07918274413809458.
[I 2025-05-11 22:42:12,434] Trial 10 finished with value: 0.07925183372755243 and parameters: {'alpha': 42.13778557747433}. Best is trial 1 w

In [73]:
# best parameters

study.best_params

{'alpha': 99.97541126726539}

In [74]:
# best value

study.best_value

0.07916409574411024